In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

c:\Users\ggsna\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [5]:
filtrados_true = pd.read_excel('../docs/respostas_danoambiental_50.xlsx')
filtrados_true = filtrados_true.loc[filtrados_true['processo_ambiental'] != False]
filtrados_true.head(10)

,Unnamed: 0,processo,processo_ambiental,justificativa,processoAnexoID,link de referencia
1,0000135-52.2021.8.16.0067,0000135-52.2021.8.16.0067,True,Trata-se de Ação Civil Pública ajuizada pelo M...,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
3,0000272-92.1992.8.26.0157,0000272-92.1992.8.26.0157,True,Ação Civil Pública - Meio Ambiente,657568057,http://jud-anexos.digesto.com.br/4f696a2b03b85...
4,0000428-83.2019.8.16.0134,0000428-83.2019.8.16.0134,True,Assunto Principal: Dano Ambiental,1088560478,http://jud-anexos.digesto.com.br/05d31619ede92...
5,0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,True,"Na petição inicial, dentre seus pedidos, os au...",1186043580,http://jud-anexos.digesto.com.br/7487b4175300b...
7,0000640-15.2017.8.11.0102,0000640-15.2017.8.11.0102,True,O Ministério Público do Estado de Mato Grosso ...,1299854564,http://jud-anexos.digesto.com.br/40b55644f0bfc...
8,0001293-11.2021.8.16.0046,0001293-11.2021.8.16.0046,True,Trata-se de “ação civil pública por dano ambie...,1462784031,http://jud-anexos.digesto.com.br/a7e7d6002b57d...
11,0002010-52.2014.8.16.0148,0002010-52.2014.8.16.0148,True,Assunto Principal: Dano Ambiental,2924793560,http://jud-anexos.digesto.com.br/bb9f51b15126e...
12,0002156-82.2011.8.16.0121,0002156-82.2011.8.16.0121,True,pode haver danos ao meio ambiente devido à det...,374401292,http://jud-anexos.digesto.com.br/f288ba0e2e7ee...
14,0002171-93.2018.8.08.0035,0002171-93.2018.8.08.0035,True,Narra o MPES que apurou a emissão de som em vi...,2583016248,http://jud-anexos.digesto.com.br/f8941539197bd...
16,0002250-66.2015.8.16.0193,0002250-66.2015.8.16.0193,True,A presente lide é abarcada pela responsabilida...,2890794934,http://jud-anexos.digesto.com.br/e57e04d13812d...


In [6]:
filtrados_com_num = pd.read_excel("../docs/processos_filtrados.xlsx")
filtrados_com_num.head(10)

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
5,5,5003677-41.2019.8.13.0521,612465137,590939497,http://jud-anexos.digesto.com.br/51644135dd7d3...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
6,6,0000527-75.2011.8.02.0020,582564055,1186042345,http://jud-anexos.digesto.com.br/b9775f943d51d...,https://www2.tjal.jus.br/pastadigital/getPDF.d...,LAUDO,2016-08-17,2024-09-12T04:13:58.579221,TJAL,LAUDO DE AVALIACAO - PAGINA 99,0000527-75.2011.8.02.0020
7,7,0011297-52.2011.8.13.0140,690025576,2589045924,http://jud-anexos.digesto.com.br/6a7cee8b71f2a...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO PERICIAL - PARTE 01 - LAUDO PERICIAL CAR...,0011297-52.2011.8.13.0140
8,8,0011297-52.2011.8.13.0140,690025576,2589045929,http://jud-anexos.digesto.com.br/0584da20569c9...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO DE PERICIA - PARTE 02 - LAUDO OFICIAL CA...,0011297-52.2011.8.13.0140
9,9,0011297-52.2011.8.13.0140,690025576,2589045772,http://jud-anexos.digesto.com.br/9d774ad2a90a2...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2024-01-25,2025-02-21T07:26:39.772134,TJMG,LAUDO - LAUDO COMPLEMENTAR CARMENSE COMERCIAL ...,0011297-52.2011.8.13.0140


In [7]:
processos_unicos = list(filtrados_com_num['num_processo'].unique())
display(filtrados_com_num.loc[filtrados_com_num['num_processo']=='Não encontrado'])
processos_unicos.remove('Não encontrado')

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
51,51,5580808-19.2022.8.09.0011,503987107,972210470,http://jud-anexos.digesto.com.br/313133a43d0b9...,https://projudi.tjgo.jus.br/BuscaProcesso?Pagi...,DECISAO,2023-01-18,2024-08-23T05:34:28.806878,TJGO,DECISAO DE PEDIDO DE URGENCIA,Não encontrado
55,55,5061164-56.2024.8.09.0051,570015135,2564299827,http://jud-anexos.digesto.com.br/2a8bb47194987...,https://projudi-2024-prd.s3.tjgo.jus.br/202412...,DECISAO,2024-12-04,2025-02-19T07:58:29.294476,TJGO,DECISAO,Não encontrado
102,102,0000257-34.2024.8.05.0110,655333385,1942855138,http://jud-anexos.digesto.com.br/30f8d4e93b94a...,https://projudi.tjba.jus.br/projudi/listagens/...,DOCUMENTO,2024-02-26,2024-11-30T22:32:58.025278,TJBA,DOCUMENTO,Não encontrado


In [83]:
def analise(texto_extraido):
    prompt = f"""
        SYSTEM: Você é meu assistente especialista em análise e extração de elementos de textos judiciais. Você irá realizar extrações especificamente sobre
        danos socioambientais de diversos tipos, pensando em futuramente usar a tabela gerada para fazer uma modelagem preditiva de multas para danos socioambientais.

        INSTRUCTIONS:
        Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
        Seu objetivo é analisar esse texto e extrair 15 informações específicas para montar um banco de dados, conforme as instruções abaixo.
        Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
        de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
        Retorno esperado (formato da resposta):
        A resposta deve ser APENAS uma string com 15 elementos separados por ";;;", na ordem específica abaixo.
        Não inclua quebras de linha, aspas, colchetes ou qualquer outro caractere além dos valores e do separador ";;;".

        [0] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
         Se não houver, retorne: "null"

        [1] Sigla da Unidade Federativa (UF) (ex: "SP", "MG" etc.)
         Se não houver, retorne: "null"

        [2] Município ou cidade do local afetado. (ex: "São Paulo", "Minas Gerais" etc)
         Se não houver, retorne: "null"

        [3] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
         Se não houver, retorne: "null"

        [4] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
         Se não houver, retorne: "null"

        [5] Tipo de impacto: Categoria do dano (ex.: "Desmatamento de APP", "Derramamento de Petróleo", "Poluição Hídrica").
        Se não houver, retorne: "null"

        [6] Descrição do impacto: Resuma o impacto em até 30 palavras.
        Se não houver, retorne: "null"

        [7] Data do impacto ambiental no formato: DD/MM/AA
         Se não houver, retorne: "null"

        [8] Extensão da área afetada (ex: "15000".)
         Se não houver, retorne: "null"
        Sempre em metros quadrados (m²) ou hectares (ha), então converta se necessário. Não use separadores para milhares

        [9] Unidade de medida da área (ex: "ha" ou "m2")
         Se não houver, retorne: "null"

        [10] Houve compensação não monetária atribuída (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
         Se houver, retorne: "True"
         Se não houver, retorne: "False"

        [11] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", "Custas Judiciais e Acordos" ou "Valoração Econômica".
         Se não houver, retorne: "null"
         Tente não fugir dessas categorias.

        [12] Tipo de multa: (0, 1 ou 2)
        Se for uma única multa aplicada sobre o responsável, coloque 0
        Se for algo como uma multa diária, coloque 1.
        Se forem os dois, como uma multa imediata e um pagamento diário, coloque 2

        [13] Valor da multa completa para o Tipo 0 ou 2: (ex: "123000.00", "999999.99")
        Caso o item 12 seja 0, complete essa coluna com o valor total da multa aplicada.
        Caso o item 12 seja 1, deixe como "null"
        Caso o item 12 seja 2, complete essa coluna com o valor total da multa imediata aplicada.

        [14] Valor da multa diária para o Tipo 1 ou 2: (ex: "123000.00", "10000.50")
        Caso o item 12 seja 1 ou 2, complete essa coluna com o valor diário a ser pago pelo responsável.
        Caso o item 12 seja 0, deixe como "null"

        IMPORTANTE: A string será usada para um dataframe posteriormente. Logo:
        NÃO inclua explicações, formatações extras, aspas ou colchetes na resposta.
        APENAS os 15 valores separados por ";;;", na ordem acima, sem repetições.
        NÃO deixe nenhuma casa em branco ou vazia. SEMPRE coloque "null" caso não encontre no texto.
        A resposta deve conter EXATAMENTE 15 elementos (de índice 0 a 14 - NADA A MAIS NEM A MENOS), na ordem definida acima.
        NÃO adicione ";;;" no final da string.
        NÃO inclua o texto original, nem explicações, apenas a string limpa.
        A ordem dos elementos PRECISA SER RESPEITADA!! NENHUM ELEMENTO PODE VIR ANTES DA SUA POSIÇÃO!!

        USER:
        Texto para análise:
        {texto_extraido}

        OUTPUT EXEMPLO:
        20°22’15.50” S e 43°25’10.30” O;;;MG;;;Mariana;;;Mineração Serra Verde Ltda.;;;Pessoa Jurídica;;;Poluição Hídrica;;;Derramamento de rejeitos de mineração contaminou 10 km do Rio Doce, afetando fauna, flora e comunidades.;;;15/03/23;;;50;;;ha;;;True;;;Obrigações de Fazer (com custo);;;2;;;500000.00;;;10000.00
    """
    
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=1.0,
    )
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    return resposta

In [ ]:
def limpa_resposta(respostas, resposta, numero_processo, processoAnexoID, link):
    lista = (resposta.text.split(";;;"))
    lista.insert(0,numero_processo)
    lista = [x for x in lista if x != '']
    lista.append(processoAnexoID)
    lista.append(link)
    
    respostas[numero_processo] = lista
    return respostas

In [85]:
respostas = {}
lista_numeros = list(filtrados_true['processo'].unique())
for processo in lista_numeros: 
    linhas_correspondentes = filtrados_true.loc[filtrados_true['processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        processoAnexoID = row['processoAnexoID']
        link = row['link de referencia']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()

        resposta_prompt = analise(texto)
        respostas = limpa_resposta(respostas,resposta_prompt,processo,processoAnexoID,link)

In [86]:
respostas

{'0000135-52.2021.8.16.0067': ['0000135-52.2021.8.16.0067',
  'null',
  'PR',
  'Cerro Azul',
  'Nataniel do Nascimento',
  'Pessoa Física',
  'Poluição',
  'Poluição não especificada',
  'null',
  'null',
  'null',
  'False',
  'null',
  '0',
  '35459.56',
  'null\n',
  2908924070,
  'http://jud-anexos.digesto.com.br/a5625273387d03675117df077783ffb2.pdf'],
 '0000272-92.1992.8.26.0157': ['0000272-92.1992.8.26.0157',
  'null',
  'SP',
  'Cubatão',
  'Petróleo Brasileiro S/A [PETROBRAS]',
  'Pessoa Jurídica',
  'Poluição Hídrica',
  'Escavações para instalação de dutos revolvendo material do fundo do rio, liberando produtos químicos e causando mortandade de peixes.',
  '25/05/92',
  'null',
  'null',
  'True',
  'Compensações Financeiras',
  '0',
  '671400.00',
  'null\n',
  657568057,
  'http://jud-anexos.digesto.com.br/4f696a2b03b85fcc919fa1bd11e03ad4.pdf'],
 '0000428-83.2019.8.16.0134': ['0000428-83.2019.8.16.0134',
  'null',
  'PR',
  'Pinhão',
  'COPEL DISTRIBUIÇÃO S.A.',
  'Pessoa 

In [82]:
respostas_teste1 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste1.columns = [
    "processo",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "topo_impacto",
    "descricao_impacto",
    "data",
    "area",
    "unidade",
    'compensacao_nao_monetaria',
    "categoria_compensacao",
    "tipo_multa",
    "valor_multa",
    "valor_multa_diaria",
    "id_documento",
    "url_documento"
]

respostas_teste1 = respostas_teste1.reset_index(drop=True)
respostas_teste1

,processo,georreferência,UF,municipio,responsavel,tipo_responsavel,topo_impacto,descricao_impacto,data,area,unidade,compensacao_nao_monetaria,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,id_documento,url_documento
0,0000135-52.2021.8.16.0067,null,PR,Cerro Azul,Nataniel do Nascimento,Pessoa Física,Poluição,Poluição não especificada no texto,null,null,null,False,null,0,35459.56,null\n,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
1,0000272-92.1992.8.26.0157,null,SP,Cubatão,Petróleo Brasileiro S/A [PETROBRAS],Pessoa Jurídica,Poluição Hídrica,Escavações para instalação de dutos revolvendo...,25/05/92,null,null,True,Compensações Financeiras,0,671400.00,null\n,657568057,http://jud-anexos.digesto.com.br/4f696a2b03b85...
2,0000428-83.2019.8.16.0134,null,PR,Pinhão,COPEL DISTRIBUIÇÃO S.A.,Pessoa Jurídica,Dano Ambiental,Dano ambiental não especificado no texto.,null,null,null,False,null,0,20000.00,null\n,1088560478,http://jud-anexos.digesto.com.br/05d31619ede92...
3,0000527-75.2011.8.02.0020,null,null,null,null,null,null,null,null,null,null,null,null,null\n,1186043580,http://jud-anexos.digesto.com.br/7487b4175300b...,None,None
4,0000640-15.2017.8.11.0102,null,MT,null,null,null,null,null,null,null,null,null,null,null\n,1299854564,http://jud-anexos.digesto.com.br/40b55644f0bfc...,None,None
5,0001293-11.2021.8.16.0046,null,PR,Arapoti,LUTE JONGSMA,Pessoa Física,Aplicação de agrotóxico,Pulverização de agrotóxico em distância inferi...,null,null,null,False,null,0,null,null\n,1462784031,http://jud-anexos.digesto.com.br/a7e7d6002b57d...
6,0002010-52.2014.8.16.0148,null,PR,Rolândia,Município de Rolândia/PR,Pessoa Jurídica,null,null,null,null,null,False,null,null,null\n,2924793560,http://jud-anexos.digesto.com.br/bb9f51b15126e...,None
7,0002156-82.2011.8.16.0121,null,PR,Nova Londrina,Município de Nova Londrina/PR,Pessoa Jurídica,null,null,null,null,null,False,null,null,null\n,374401292,http://jud-anexos.digesto.com.br/f288ba0e2e7ee...,None
8,0002171-93.2018.8.08.0035,null,ES,Vila Velha,A Igreja,Pessoa Jurídica,Poluição Sonora,Emissão de ruídos acima dos limites permitidos...,null,null,null,False,null,2,1000.00,500.00\n,2583016248,http://jud-anexos.digesto.com.br/f8941539197bd...
9,0002250-66.2015.8.16.0193,null,PR,Colombo,Imperador Projetos e Construções Ltda.,Pessoa Jurídica,null,null,null,null,null,False,null,null,null\n,2890794934,http://jud-anexos.digesto.com.br/e57e04d13812d...,None
